In [1]:
import datetime, time, copy, json, requests
from IPython.display import Image

from squid_py.config import Config
from squid_py.ocean.ocean import Ocean

from ocean_cli.ocean import get_default_account

In [2]:
config_bob = Config(filename='./bob.ini')
ocean = Ocean(config_bob)
bob = get_default_account(config_bob)

config_alice = Config(filename='./alice.ini')
ocean_alice = Ocean(config_alice)
alice = get_default_account(config_alice)

2019-06-20 16:27:10 dimi-xps squid_py.keeper.diagnostics[4474] INFO Keeper contract artifacts (JSON abi files) at: /home/dimi/oceanprotocol/ocean-cli-py/artifacts
2019-06-20 16:27:10 dimi-xps squid_py.keeper.diagnostics[4474] INFO Using keeper contracts from network spree, network id is 8996
2019-06-20 16:27:10 dimi-xps squid_py.keeper.diagnostics[4474] INFO Looking for keeper contracts ending with ".spree.json", e.g. "AgreementStoreManager.spree.json".
2019-06-20 16:27:10 dimi-xps root[4474] INFO Finished loading keeper contracts:
	Dispenser: 0xF152cF3c67dFD41a317eAe8fAc0e1e8E98724A13
	OceanToken: 0x726baA2f854A3BEC2378a707AeB38c9d933Ebad6
	DIDRegistry: 0xc354ba9AD5dF1023C2640b14A09E61a500F21546
	AgreementStoreManager: 0x645439117eB378a6d35148452E287a038666Ed67
	TemplateStoreManager: 0xfeA10BBb093d7fcb1EDf575Aa7e28d37b9DcFcE9
	ConditionStoreManager: 0x336EFb3c9E56F713dFdA4CDB3Dd0882F3226b6eE
	AccessSecretStoreCondition: 0x19513460bc16254c74AE806683E906478A42B543
	SignCondition: 0xEEE5

In [3]:
ocean.tokens.request(bob, 1)
print(ocean.accounts.balance(alice))
print(ocean.accounts.balance(bob))

Balance(eth=999999999999999947796880000, ocn=306)
Balance(eth=1000000000000000294214450000, ocn=27999999999999999571)


In [4]:
from ocean_cli.api.assets import list_assets, search

assets = list_assets(ocean, bob, None)
print(assets)
print(search(ocean, 'jupyter', pretty=True))
print(ocean.assets.resolve(assets[0]).as_dictionary())

2019-06-20 16:27:12 dimi-xps ocean[4474] INFO Searching asset containing: jupyter


['did:op:c8703666851442acaeb443936ee990101ddc8c0c8b954a138f5f3fbc761a8219', 'did:op:780a0f2ef5b34690bd935edbc48941aaeb3f97d7f2514b538ca63528bb1f1113', 'did:op:040754de401e4c6abd25b40b03d5e78e65d202d52c154a6d9e05bc49f2b95e8a', 'did:op:7f7e09243fcf4540beb21a69eca41b63b0e8c90c68ad479f99d431c8057cc595', 'did:op:336bfaa70c5f4761bb4cb02c299aeadd1680951c84d94a2a952d0e4f32ecf352', 'did:op:f6cbd3871c3148109919b05133703addf76b2970e20845deb495b0eec5924cc4', 'did:op:f3912e94961c4246923982f13561b61655ec3e102f8241d98630a4d58ffcaca0', 'did:op:6e381a44ccec452aabb785cdfd0e9dc0da5ab0011fb542bb95e950686ce46547', 'did:op:0e86da194fa54d75a2615da4effac25387c2a861afcb4a99bff0a143acea06fb', 'did:op:b99c6929622b4c5ca08f4b001f48657409385499a5b74fdaa67e37bbb28fa344', 'did:op:cdd86dc611cb445aba9e393620bf5fd612e2c92f53a54f0d928ccec4e473866c', 'did:op:47f8fa45c4db431db8ca49849760c1cc1bc26fb1f0fb47778a03fc1d45fe8311', 'did:op:f9232f50b1184ea8aff2d51700982c394d50fdf5349f405dba7443e52a05911e', 'did:op:d0e0e679e35a4273

In [5]:
from ocean_cli.api.assets import create

def make_metadata(name, url, price):
    name = f'{name}:{datetime.datetime.now()}'
    price = 12
    return {
      "base": {
        "name": name,
        "dateCreated": str(datetime.datetime.now()),
        "author": "Luigi",
        "license": "CC0: Public Domain",
        "price": price,
        "files": [{
            "url": url
          }
        ],
        "type": "dataset"
      }
    }

price = 0
did_localhost = create(ocean, bob, 
                       make_metadata('readme', 'README.md', price), 
                       secret_store=True, 
                       price=price)
print(did_localhost)

2019-06-20 16:27:12 dimi-xps ocean[4474] INFO Content urls encrypted successfully 0xb092b352f0a060ac2c758e46cb81ca48b29b7fccd111e897584a83a933f6a322edc7714e1f71


cb8388450dee4b4b819110bcb7232213ea4ea4ec6e2f4f9a94456d3f8ea69a52 [{"url": "README.md"}] 0
0xcd20551c52ea9e39fd356cd538c111cb1c9dca87f8d830dabe42ef1708db6e4f374d3335d62f9ebf673e2b6213614a305e807e18e61ac185096814b6a7be5eb700
0xb092b352f0a060ac2c758e46cb81ca48b29b7fccd111e897584a83a933f6a322edc7714e1f71


2019-06-20 16:27:15 dimi-xps ocean[4474] INFO DDO with DID did:op:cb8388450dee4b4b819110bcb7232213ea4ea4ec6e2f4f9a94456d3f8ea69a52 successfully registered on chain.


did:op:cb8388450dee4b4b819110bcb7232213ea4ea4ec6e2f4f9a94456d3f8ea69a52


In [6]:
from ocean_cli.api.assets import order_consume

order_localhost = order_consume(ocean, bob, did_localhost, 'get')
print(order_localhost['response'],
      ocean.agreements.status(order_localhost['agreement']),
      ocean.accounts.balance(alice),
      ocean.accounts.balance(bob))

2019-06-20 16:27:15 dimi-xps escrowAccessSecretStoreTemplate[4474] INFO Creating agreement 0x93853e74e96e48149eafc60a88b2491544227b967bb24b338d78731bfd04d558 with did=0xcb8388450dee4b4b819110bcb7232213ea4ea4ec6e2f4f9a94456d3f8ea69a52, consumer=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e.
2019-06-20 16:27:18 dimi-xps ocean[4474] INFO Service agreement 0x93853e74e96e48149eafc60a88b2491544227b967bb24b338d78731bfd04d558 created successfully.


# Ocean Command Line Interface (Python)

> CLI tool for Ocean Protocol
> [oceanprotocol.com](https://oceanprotocol.com)

---

**ð²ð¦ THERE BE DRAGONS AND SQUIDS. This is in alpha state and you can expect running into problems. If you run into them, please open up [a new issue](https://github.com/oceanprotocol/tuna/issues). ð¦ð²**

---

## Table of Contents

  - [Get started](#get-started)
  - [License](#license)

---

## Get started

### Setup
> The setup assumes either running a local network or setup a configuration file
for connecting to Nile, Duero, ...


## Compatibility

- keeper-contracts: `v0.9.7`
- brizo: `>0.3.7`

```bash
virtualenv venv -p python3
source venv/bin/activate
python setup.py install
```

### Usage

We're assuming user config files:
- Default: `config.ini`
- Alice: `alice.ini`
- Bob: `bob.ini`

```bash
## basics
> ocean --help

## accounts and tokens
> ocean accounts list
> ocean accounts balance
# switch accounts
> ocean -c <config.ini> accounts balance


In [7]:
from ocean_cli.api.assets import order, decrypt

# publish json
url = 'https://api.coingecko.com/api/v3/simple/price?ids=ocean-protocol&vs_currencies=EUR%2CUSD'
did_json = create(ocean, bob, 
                  make_metadata('json', url, price=len(url)), 
                  secret_store=True, price=len(url), service_endpoint='/')

# consume json
agreement_id = order(ocean, bob, did_json)
print(json.dumps(
    requests.get(decrypt(ocean, bob, did_json)[0]['url']).json(),
    indent=2, sort_keys=True))

ccdc9f1db6784242a3ad969b8699f2537768ae1cd21a4c97a95c859c4bf16b8d [{"url": "https://api.coingecko.com/api/v3/simple/price?ids=ocean-protocol&vs_currencies=EUR%2CUSD"}] 0
0x21ba85500fe8a3fff7f4f744b2a7f6f286a2a8d2c16066aa108c94fa823e949e30b8261a36af7f2ba9ef2875976d8b4b3cbfa336ac7363570ef3ecbe89ba831c00


2019-06-20 16:27:24 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: http://localhost:8000?url=README.md&serviceAgreementId=0x93853e74e96e48149eafc60a88b2491544227b967bb24b338d78731bfd04d558&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:24 dimi-xps squid_py.brizo.brizo[4474] WARNING It was not possible to get the file name. expected string or bytes-like object
2019-06-20 16:27:24 dimi-xps ocean[4474] INFO Content urls encrypted successfully 0x97cefcc93dff10e57ece6ab591dacfa80de6755bf233fa076642568651ff195e12bb0fb259c005e6e569018764060b0c578bc552319520ca6fb848d4499e759a36df115af9ae7944db2c81f0e29fc66046689cfe45c26b7c864d34d34b449944b38d572a51753939a1ed1d1b0fb9da184fab90e48b


0x97cefcc93dff10e57ece6ab591dacfa80de6755bf233fa076642568651ff195e12bb0fb259c005e6e569018764060b0c578bc552319520ca6fb848d4499e759a36df115af9ae7944db2c81f0e29fc66046689cfe45c26b7c864d34d34b449944b38d572a51753939a1ed1d1b0fb9da184fab90e48b


2019-06-20 16:27:25 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: http://localhost:8000?url=README.md&serviceAgreementId=0x93853e74e96e48149eafc60a88b2491544227b967bb24b338d78731bfd04d558&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:25 dimi-xps squid_py.brizo.brizo[4474] WARNING It was not possible to get the file name. expected string or bytes-like object
2019-06-20 16:27:26 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: http://localhost:8000?url=README.md&serviceAgreementId=0x93853e74e96e48149eafc60a88b2491544227b967bb24b338d78731bfd04d558&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:26 dimi-xps squid_py.brizo.brizo[4474] WARNING It was not possible to get the file name. expected string or bytes-like object
2019-06-20 16:27:26 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: http://localhost:8000?url=README.md&serviceAgreem

{
  "ocean-protocol": {
    "eur": 0.02840987,
    "usd": 0.03209799
  }
}


In [8]:
# publish gif
url = 'https://i.giphy.com/media/3oEduQAsYcJKQH2XsI/giphy.webp'
did_created = create(ocean, bob, 
                     make_metadata('gif', url, price=len(url)), 
                     secret_store=True, price=len(url), service_endpoint='/')

# consume gif
agreement_id = order(ocean, bob, did_created)
Image(url=decrypt(ocean, bob, did_created)[0]['url'])
    

fa50af3541ed4453bea5b2b6fde5e88e896437708db2447ba2ca51b99ecea572 [{"url": "https://i.giphy.com/media/3oEduQAsYcJKQH2XsI/giphy.webp"}] 0
0xe974d3b3a65ad7de6ac9e029aee55d47c84ade1b3e2b16d83c2242e0394cb95c6b7a663a824692487ec644e48dd769573eac79b2a209c01fad6dc7367c10846b00
0xb3e1ec60b6d50a89cc0e6a14a8058d9261ad4feee1152aa7bb9ce2540772ed5d80988184d59365f9c9cd1febab5dbf6c30556c6da08df30c873edc94648380f81d118b3e9ba877a3f74b63a5defa070fd21b86e4


2019-06-20 16:27:37 dimi-xps ocean[4474] INFO Content urls encrypted successfully 0xb3e1ec60b6d50a89cc0e6a14a8058d9261ad4feee1152aa7bb9ce2540772ed5d80988184d59365f9c9cd1febab5dbf6c30556c6da08df30c873edc94648380f81d118b3e9ba877a3f74b63a5defa070fd21b86e4
2019-06-20 16:27:37 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: /?url=https://api.coingecko.com/api/v3/simple/price?ids=ocean-protocol&vs_currencies=EUR%2CUSD&serviceAgreementId=0x3dfa193c2baf4e9a9af8bf590f3c2f94f6b2be2f3aa14a90acd44138a374f741&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:38 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: /?url=https://api.coingecko.com/api/v3/simple/price?ids=ocean-protocol&vs_currencies=EUR%2CUSD&serviceAgreementId=0x3dfa193c2baf4e9a9af8bf590f3c2f94f6b2be2f3aa14a90acd44138a374f741&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:39 dimi-xps ocean[4474] INFO DDO with DID did:op

In [ ]:
import pandas as pd

# publish csv
url = 'https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv'
did_created = create(ocean, bob,
                     make_metadata('csv', url, price=len(url)), 
                     secret_store=True, price=len(url), service_endpoint='/')

# consume csv
agreement_id = order(ocean, bob, did_created)
df = pd.read_csv(decrypt(ocean, bob, did_created)[0]['url'])
df.describe()

2019-06-20 16:27:48 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: /?url=https://i.giphy.com/media/3oEduQAsYcJKQH2XsI/giphy.webp&serviceAgreementId=0x21795bcbd71d4758854304542b6d0d5992968490c4f24554b9874af9271c9e06&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:48 dimi-xps ocean[4474] INFO Content urls encrypted successfully 0xceb1ecaec5097f3de472016c52f1ff4326332a3cecc0ae8dd6c17a523e4f93efe5182187c2344c8f1501266114e11d21ee43af4a81b58b02890c0374dec370c3a67b0e3b0de8939c808d95c14d3b018605ed2857807270be576e89ff7c32c676f7d5f9e57d69b644c9


50335b220a204357a1df18e40027851b4d2cf28e956b4a308c4a81a74b1bbfc2 [{"url": "https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv"}] 0
0x4d84ea714c59df8f17486eebe5a9773223d4ee5ceaafc657fb942ecfa9093c3e69b31a6b6bb2c5bfc86916010d130dc3be01e0e16f71072d0013a3259709a48f00
0xceb1ecaec5097f3de472016c52f1ff4326332a3cecc0ae8dd6c17a523e4f93efe5182187c2344c8f1501266114e11d21ee43af4a81b58b02890c0374dec370c3a67b0e3b0de8939c808d95c14d3b018605ed2857807270be576e89ff7c32c676f7d5f9e57d69b644c9


2019-06-20 16:27:50 dimi-xps squid_py.brizo.brizo[4474] INFO invoke purchase endpoint with this url: /?url=https://i.giphy.com/media/3oEduQAsYcJKQH2XsI/giphy.webp&serviceAgreementId=0x21795bcbd71d4758854304542b6d0d5992968490c4f24554b9874af9271c9e06&consumerAddress=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e
2019-06-20 16:27:51 dimi-xps ocean[4474] INFO DDO with DID did:op:50335b220a204357a1df18e40027851b4d2cf28e956b4a308c4a81a74b1bbfc2 successfully registered on chain.
2019-06-20 16:27:51 dimi-xps escrowAccessSecretStoreTemplate[4474] INFO Creating agreement 0x33325fd451b7403297b06d9740abb537836c4c99b8644e73ae7c8fc3197c216d with did=0x50335b220a204357a1df18e40027851b4d2cf28e956b4a308c4a81a74b1bbfc2, consumer=0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e.


In [ ]:
# publish api
url = 'http://localhost:8080'
token = '?secret=muchwow'
did_api = create(ocean, bob, 
                  make_metadata('json', token, price=len(url)), 
                  secret_store=True, price=len(url), service_endpoint=url)

# consume api
json.loads(order_consume(ocean, bob, did_api, 'get')['response'])

In [ ]:
ocean.assets.resolve(did_api).as_dictionary()